In [36]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization,Flatten,Conv1D,MaxPooling1D,Dropout
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import zipfile
from numpy import array
from numpy import hstack

In [37]:
blocks = [1030, 2030, 3030]
#blocks = [1030, 2030, 3030, 4030, 5030, 6030, 7030]

dfss = []
for block in blocks:
    print(block)
    dfss.append(pd.read_csv('combined_df_'+str(block)+'.csv'))

1030
2030
3030


In [39]:
df = pd.concat(dfss)
df.shape

(1685106, 71)

In [40]:
def get_dataset():
    X = df[['Leading angle','Side tilt angle','Tool Tip Point X',
           'Tool Tip Point Y', 'Tool Tip Point Z', 'Tool Orientation X',
           'Tool Orientation Y', 'Tool Orientation Z',]].copy(deep=True).to_numpy()
    y = df[['MachineX', 'MachineY', 'MachineZ', 'MachineA', 'MachineC']].copy(deep=True).to_numpy()
    
    return X,y

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [43]:
X,y = get_dataset()
n_inputs = X.shape[1]
n_outputs = y.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train, X_test, y_train, y_test = X_train[0:1000,:], X_test[0:1000,:], y_train[0:1000,:], y_test[0:1000,:]

In [44]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(8, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(Dense(units=hp.Int("units_" + str(i), min_value=20, max_value=1000, step=20,default=160), activation=hp.Choice('dense_activation',values=['relu', 'tanh', 'sigmoid'],default='relu')))
    
    model.add(Dense(n_outputs))
    
    model.compile(optimizer= tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)), loss=tf.keras.losses.MeanSquaredError() )  
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=150, mode='min')
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)


In [45]:
tuner = kt.RandomSearch(build_model,objective='val_loss',max_trials=100,executions_per_trial=1,directory='random_search', project_name='ANN')
tuner.search(X_train, y_train, validation_data=(X_test, y_test),callbacks=[early_stopping,model_checkpoint],epochs=2000,batch_size = 10)

tuner.search_space_summary()
                  
tuner.results_summary()
models = tuner.get_best_models(num_models=1)
best_model = models[0]
loss = best_model.evaluate(X_test, y_test)  

history = best_model.fit(X_train, y_train, epochs=2000,validation_data=(X_test,y_test), verbose=0,callbacks=[early_stopping,model_checkpoint], batch_size = 10)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')

plt.legend()
plt.show()

model.summary()
print(loss)

Trial 19 Complete [00h 01m 24s]
val_loss: 2386.947265625

Best val_loss So Far: 2198.466796875
Total elapsed time: 00h 49m 41s

Search: Running Trial #20

Hyperparameter    |Value             |Best Value So Far 
num_layers        |7                 |4                 
units_0           |220               |80                
dense_activation  |sigmoid           |relu              
learning_rate     |0.0015534         |0.00099064        
units_1           |80                |620               
units_2           |740               |960               
units_3           |900               |920               
units_4           |740               |400               
units_5           |140               |680               
units_6           |40                |540               
units_7           |560               |540               
units_8           |100               |580               

Epoch 1/2000
100/100 [==============================] - 1s 6ms/step - loss: 16915.4512 - val_loss: 1662

KeyboardInterrupt: 